In [1]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

plt.rcParams['figure.figsize'] = [4, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

# The emergence of band structures

Yifei Guan (yifei.guan@epfl.ch)

This jupyter notebook is intended to provide a illustration of the band structures in solides, starting from the points of view of free electrons and atomic orbitals. The goals are:
- To understand how the Bloch theorem leads to band structures.
- To see how the electronic bands emerge from the interplay of periodic potential and kinetic energy.
- Understanding the relationship between the Schrodinger equation and the calculation*

## The Bloch theorem
In solids, the lattice has discrete translational symmetry. The Bloch theorem says that, such symmetry applies to the wavefunction as well, such that $$[H,R_i]=0$$
To study the energy levels in solides, one can adapt the simplest Hamiltonian:$$ H = -\frac{h^2}{2m} \nabla^2 + U(r)$$ where $U(r+R_i)=U(r)$   



From the commutation relations of the Hamiltonian, the wavefunctions should be of the form $\psi(nR_i + r) = \psi(r) \exp(-inR_i)$ which is called Bloch functions.

## From the atomic orbitals/free electrons to band structures

The Hamiltonian $$ H = -\frac{h^2}{2m} \nabla^2 + U(r)$$ contains two components: the kinetic energy $E_k=-\frac{h^2}{2m} \nabla^2$ and the potential $U$. Thus, we can look into the formation of band structure from the limits of free electrons($E_k >> U$) or localized orbitals($E_k<<U$) and treating the other term as a perturbation.

In [2]:
def interactive_bands(t,g):
    '''
    :param k: wave number
    '''

    nq=30
    # Define the position of the original lattice
    qs=np.arange(nq)
    plx=np.zeros((nq+1,nq+1))
    ply=np.zeros((nq+1,nq+1))
    #plz=np.zeros((nq+1,nq+1))
    z=1j*3.14/nq
    mat=np.zeros((nq+1,nq+1),dtype=complex)
    #print(z)
    #print(z*z)
    for i in range(0,nq+1):
        y=np.ones(nq)*t
        mat=np.zeros((nq+1,nq+1),dtype=complex)
        mat=mat+np.diag(y,-1)+np.diag(y,1)# nearest neibour positions
        mat[nq,0]=np.exp(2*z*(i-nq/1))*t
        mat[0,nq]=np.exp(-2*z*(i-nq/1))*t
        mat[0,0]=g
        #mat=mat+np.diag(np.arange(t,nq+1))
        
        #print(np.exp(-z*i)*t)

        ply[i]=np.linalg.eigvalsh(mat)#+t*2
        #print(ply[i-1])
        plx[i]=np.ones(nq+1)*(i-nq/2)
        #plz[i]=t*(((i-nq/2)/nq/2)**2)-2*t
        #plt.plot(ply[i-1])
    
    for i in range(0,nq+1):
        plt.plot(plx[:,i],ply[:,i],'b')
        plt.plot(plx[:,i]+nq,ply[:,i],'g')
        plt.plot(plx[:,i]-nq,ply[:,i],'r')
        #plt.plot(plx[:,i]-nq*2,ply[:,i])
    x = np.linspace(-1.5*nq,1.5*nq,100)
    plz=t*(6.28*x/(nq*nq))**2 - 2*t
    plt.plot(x,plz,'y')
    
    ax = plt.gca()
    #ax.set_aspect(90)
    plt.ylim(-2*t,-t)
    zz=ply[0,0]+2*t
    print(zz)
    tic=-2*t+zz*np.arange(1,7)**2
    plt.yticks(tic)
    plt.show()

## An interactive look

In the following block, there is a simple example on how band structure emerges in lattices. The potential function $U$ takes a simplest form $U=\delta (x)$




By tuning the parameter t and g, one can travel between the periodic free electronic limit and the discrete energy levels. One can see that, at the limit $t<<g$, the spectrum resembles a series of discrete energy levels, which are rarely dependent on the wave vector $k$. Such a spectrum reflect the fact that the eigenstates are mostly localized by the strong potential. On the other hand, if we take $t$ as the dominant term, the spectrum then looks like a series of quadratic functions. This is consistent with the fact that the system is periodic and the electrons are nearly free. 

In between the two cases, there then arrives our band structures: the spectrum forms several energy bands while each band has some dispersion. 

In [3]:
interact(interactive_bands,t=(0.01,1.00,0.05),g=(0,1.00,0.05))

interactive(children=(FloatSlider(value=0.46, description='t', max=1.0, min=0.01, step=0.05), FloatSlider(valu…

<function __main__.interactive_bands(t, g)>